# Character-wise RNN to use as Tensorboard Example

In this notebook, I'll build a character-wise RNN trained on "Anna Karenina". It'll be able to generate new text based on the text from the book. The network's graph, and parameters will be visualized and debugged using Tensorboard

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [15]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [16]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [17]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [18]:
chars[:100]

array([77, 51, 46, 35, 28, 18, 48, 53, 20, 58, 58, 58, 45, 46, 35, 35, 52,
       53, 23, 46, 50, 67, 72, 67, 18,  8, 53, 46, 48, 18, 53, 46, 72, 72,
       53, 46, 72, 67, 17, 18, 68, 53, 18, 44, 18, 48, 52, 53,  1, 37, 51,
       46, 35, 35, 52, 53, 23, 46, 50, 67, 72, 52, 53, 67,  8, 53,  1, 37,
       51, 46, 35, 35, 52, 53, 67, 37, 53, 67, 28,  8, 53,  6, 33, 37, 58,
       33, 46, 52, 39, 58, 58, 60, 44, 18, 48, 52, 28, 51, 67, 37], dtype=int32)

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [19]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [20]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [21]:
train_x.shape

(10, 178400)

In [22]:
train_x[:,:10]

array([[77, 51, 46, 35, 28, 18, 48, 53, 20, 58],
       [69, 37, 19, 53, 51, 18, 53, 50,  6, 44],
       [53, 24, 46, 28, 24, 51, 67, 37, 63, 53],
       [ 6, 28, 51, 18, 48, 53, 33,  6,  1, 72],
       [53, 28, 51, 18, 53, 72, 46, 37, 19, 55],
       [53,  5, 51, 48,  6,  1, 63, 51, 53, 72],
       [28, 53, 28,  6, 58, 19,  6, 39, 58, 58],
       [ 6, 53, 51, 18, 48,  8, 18, 72, 23,  0],
       [51, 46, 28, 53, 67,  8, 53, 28, 51, 18],
       [18, 48,  8, 18, 72, 23, 53, 46, 37, 19]], dtype=int32)

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [23]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [24]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope("inputs"):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')

    
    with tf.name_scope("targets"):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_layers"):
        cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper( tf.contrib.rnn.BasicLSTMCell(lstm_size)) for _ in range(num_layers)])

    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope("sequence_reshape"):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN putputs to a softmax layer and calculate the cost
    with tf.name_scope("logits"):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b
        tf.summary.histogram("softmax_w",softmax_w)
        tf.summary.histogram("softmax_b",softmax_b)

    with tf.name_scope("predictions"):
        preds = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram("predictions",preds)
    
    with tf.name_scope("cost"):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar("cost",cost)

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope("train"):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
        
    summaries = tf.summary.merge_all()

    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer','summaries']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [25]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Write out the graph for TensorBoard

In [26]:
model = build_rnn(len(vocab),
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter("./logs/train",sess.graph)
    test_writer = tf.summary.FileWriter("./logs/test")

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'dict' object has no attribute 'name'


## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [27]:
!mkdir -p checkpoints/anna

In [28]:
epochs = 1
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state}
            summaries,batch_loss, new_state, _ = sess.run([model.summaries,model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            train_writer.add_summary(summaries,iteration)
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    summaries,batch_loss, new_state = sess.run([model.summaries,model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                test_writer.add_summary(summaries,iteration)
                
                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/anna/i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/1  Iteration 1/178 Training loss: 4.4165 0.1873 sec/batch
Epoch 1/1  Iteration 2/178 Training loss: 4.3668 0.1474 sec/batch
Epoch 1/1  Iteration 3/178 Training loss: 4.1579 0.1476 sec/batch
Epoch 1/1  Iteration 4/178 Training loss: 4.2689 0.1492 sec/batch
Epoch 1/1  Iteration 5/178 Training loss: 4.1726 0.1477 sec/batch
Epoch 1/1  Iteration 6/178 Training loss: 4.0751 0.1513 sec/batch
Epoch 1/1  Iteration 7/178 Training loss: 3.9772 0.1495 sec/batch
Epoch 1/1  Iteration 8/178 Training loss: 3.8872 0.1477 sec/batch
Epoch 1/1  Iteration 9/178 Training loss: 3.8096 0.1510 sec/batch
Epoch 1/1  Iteration 10/178 Training loss: 3.7503 0.1478 sec/batch
Epoch 1/1  Iteration 11/178 Training loss: 3.7003 0.1495 sec/batch
Epoch 1/1  Iteration 12/178 Training loss: 3.6585 0.1474 sec/batch
Epoch 1/1  Iteration 13/178 Training loss: 3.6211 0.1476 sec/batch
Epoch 1/1  Iteration 14/178 Training loss: 3.5895 0.1471 sec/batch
Epoch 1/1  Iteration 15/178 Training loss: 3.5605 0.1476 sec/batch
Epoc

Epoch 1/1  Iteration 125/178 Training loss: 3.1087 0.1485 sec/batch
Epoch 1/1  Iteration 126/178 Training loss: 3.1059 0.1492 sec/batch
Epoch 1/1  Iteration 127/178 Training loss: 3.1033 0.1477 sec/batch
Epoch 1/1  Iteration 128/178 Training loss: 3.1007 0.1477 sec/batch
Epoch 1/1  Iteration 129/178 Training loss: 3.0980 0.1476 sec/batch
Epoch 1/1  Iteration 130/178 Training loss: 3.0952 0.1479 sec/batch
Epoch 1/1  Iteration 131/178 Training loss: 3.0926 0.1489 sec/batch
Epoch 1/1  Iteration 132/178 Training loss: 3.0895 0.1493 sec/batch
Epoch 1/1  Iteration 133/178 Training loss: 3.0867 0.1479 sec/batch
Epoch 1/1  Iteration 134/178 Training loss: 3.0837 0.1569 sec/batch
Epoch 1/1  Iteration 135/178 Training loss: 3.0805 0.1576 sec/batch
Epoch 1/1  Iteration 136/178 Training loss: 3.0772 0.1568 sec/batch
Epoch 1/1  Iteration 137/178 Training loss: 3.0741 0.1564 sec/batch
Epoch 1/1  Iteration 138/178 Training loss: 3.0711 0.1534 sec/batch
Epoch 1/1  Iteration 139/178 Training loss: 3.07

### Hyperparameter selection
Tensorboard can be used for hyperparameter selection, to show this, i will create a train_network function that takes a model, an epoch number and a file_writer and trains(and stores tensorboard summaries) data about the model with that selected hyperparameters,then we use tensorboard to compare results

In [35]:
def train(model, epochs, file_writer):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Use the line below to load a checkpoint and resume training
        #saver.restore(sess, 'checkpoints/anna20.ckpt')

        n_batches = int(train_x.shape[1]/num_steps)
        iterations = n_batches * epochs
        for e in range(epochs):

            # Train network
            new_state = sess.run(model.initial_state)
            loss = 0
            for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
                iteration = e*n_batches + b
                start = time.time()
                feed = {model.inputs: x,
                        model.targets: y,
                        model.keep_prob: 0.5,
                        model.initial_state: new_state}
                summary, batch_loss, new_state, _ = sess.run([model.summaries, model.cost, 
                                                              model.final_state, model.optimizer], 
                                                              feed_dict=feed)
                loss += batch_loss
                end = time.time()
                print('Epoch {}/{} '.format(e+1, epochs),
                      'Iteration {}/{}'.format(iteration, iterations),
                      'Training loss: {:.4f}'.format(loss/b),
                      '{:.4f} sec/batch'.format((end-start)))

                file_writer.add_summary(summary, iteration)

In [ ]:
epochs = 20
batch_size = 100
num_steps = 100
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

for lstm_size in [128,256,512]:
    for num_layers in [1, 2]:
        for learning_rate in [0.002, 0.001]:
            log_string = 'logs/hp_selection/lr={},rl={},ru={}'.format(learning_rate, num_layers, lstm_size)
            writer = tf.summary.FileWriter(log_string)
            model = build_rnn(len(vocab), 
                    batch_size=batch_size,
                    num_steps=num_steps,
                    learning_rate=learning_rate,
                    lstm_size=lstm_size,
                    num_layers=num_layers)
            
            train(model, epochs, writer)

Epoch 1/20  Iteration 1/3560 Training loss: 4.4229 0.0392 sec/batch
Epoch 1/20  Iteration 2/3560 Training loss: 4.4120 0.0256 sec/batch
Epoch 1/20  Iteration 3/3560 Training loss: 4.3998 0.0267 sec/batch
Epoch 1/20  Iteration 4/3560 Training loss: 4.3825 0.0269 sec/batch
Epoch 1/20  Iteration 5/3560 Training loss: 4.3489 0.0259 sec/batch
Epoch 1/20  Iteration 6/3560 Training loss: 4.2688 0.0261 sec/batch
Epoch 1/20  Iteration 7/3560 Training loss: 4.1706 0.0256 sec/batch
Epoch 1/20  Iteration 8/3560 Training loss: 4.0814 0.0279 sec/batch
Epoch 1/20  Iteration 9/3560 Training loss: 4.0002 0.0281 sec/batch
Epoch 1/20  Iteration 10/3560 Training loss: 3.9280 0.0278 sec/batch
Epoch 1/20  Iteration 11/3560 Training loss: 3.8621 0.0258 sec/batch
Epoch 1/20  Iteration 12/3560 Training loss: 3.8055 0.0276 sec/batch
Epoch 1/20  Iteration 13/3560 Training loss: 3.7563 0.0258 sec/batch
Epoch 1/20  Iteration 14/3560 Training loss: 3.7144 0.0271 sec/batch
Epoch 1/20  Iteration 15/3560 Training loss

Epoch 1/20  Iteration 124/3560 Training loss: 3.0748 0.0354 sec/batch
Epoch 1/20  Iteration 125/3560 Training loss: 3.0723 0.0351 sec/batch
Epoch 1/20  Iteration 126/3560 Training loss: 3.0696 0.0353 sec/batch
Epoch 1/20  Iteration 127/3560 Training loss: 3.0671 0.0407 sec/batch
Epoch 1/20  Iteration 128/3560 Training loss: 3.0647 0.0372 sec/batch
Epoch 1/20  Iteration 129/3560 Training loss: 3.0621 0.0350 sec/batch
Epoch 1/20  Iteration 130/3560 Training loss: 3.0596 0.0350 sec/batch
Epoch 1/20  Iteration 131/3560 Training loss: 3.0572 0.0410 sec/batch
Epoch 1/20  Iteration 132/3560 Training loss: 3.0546 0.0440 sec/batch
Epoch 1/20  Iteration 133/3560 Training loss: 3.0521 0.0367 sec/batch
Epoch 1/20  Iteration 134/3560 Training loss: 3.0496 0.0353 sec/batch
Epoch 1/20  Iteration 135/3560 Training loss: 3.0467 0.0427 sec/batch
Epoch 1/20  Iteration 136/3560 Training loss: 3.0441 0.0367 sec/batch
Epoch 1/20  Iteration 137/3560 Training loss: 3.0414 0.0350 sec/batch
Epoch 1/20  Iteratio

Epoch 2/20  Iteration 242/3560 Training loss: 2.4403 0.0379 sec/batch
Epoch 2/20  Iteration 243/3560 Training loss: 2.4390 0.0428 sec/batch
Epoch 2/20  Iteration 244/3560 Training loss: 2.4383 0.0381 sec/batch
Epoch 2/20  Iteration 245/3560 Training loss: 2.4373 0.0379 sec/batch
Epoch 2/20  Iteration 246/3560 Training loss: 2.4359 0.0380 sec/batch
Epoch 2/20  Iteration 247/3560 Training loss: 2.4347 0.0386 sec/batch
Epoch 2/20  Iteration 248/3560 Training loss: 2.4339 0.0448 sec/batch
Epoch 2/20  Iteration 249/3560 Training loss: 2.4329 0.0428 sec/batch
Epoch 2/20  Iteration 250/3560 Training loss: 2.4321 0.0382 sec/batch
Epoch 2/20  Iteration 251/3560 Training loss: 2.4312 0.0402 sec/batch
Epoch 2/20  Iteration 252/3560 Training loss: 2.4302 0.0397 sec/batch
Epoch 2/20  Iteration 253/3560 Training loss: 2.4292 0.0380 sec/batch
Epoch 2/20  Iteration 254/3560 Training loss: 2.4288 0.0410 sec/batch
Epoch 2/20  Iteration 255/3560 Training loss: 2.4278 0.0373 sec/batch
Epoch 2/20  Iteratio

Epoch 3/20  Iteration 361/3560 Training loss: 2.2407 0.0436 sec/batch
Epoch 3/20  Iteration 362/3560 Training loss: 2.2386 0.0400 sec/batch
Epoch 3/20  Iteration 363/3560 Training loss: 2.2391 0.0390 sec/batch
Epoch 3/20  Iteration 364/3560 Training loss: 2.2402 0.0407 sec/batch
Epoch 3/20  Iteration 365/3560 Training loss: 2.2411 0.0408 sec/batch
Epoch 3/20  Iteration 366/3560 Training loss: 2.2409 0.0396 sec/batch
Epoch 3/20  Iteration 367/3560 Training loss: 2.2391 0.0414 sec/batch
Epoch 3/20  Iteration 368/3560 Training loss: 2.2383 0.0390 sec/batch
Epoch 3/20  Iteration 369/3560 Training loss: 2.2383 0.0401 sec/batch
Epoch 3/20  Iteration 370/3560 Training loss: 2.2401 0.0392 sec/batch
Epoch 3/20  Iteration 371/3560 Training loss: 2.2396 0.0387 sec/batch
Epoch 3/20  Iteration 372/3560 Training loss: 2.2390 0.0402 sec/batch
Epoch 3/20  Iteration 373/3560 Training loss: 2.2387 0.0405 sec/batch
Epoch 3/20  Iteration 374/3560 Training loss: 2.2402 0.0393 sec/batch
Epoch 3/20  Iteratio

Epoch 3/20  Iteration 479/3560 Training loss: 2.1846 0.0404 sec/batch
Epoch 3/20  Iteration 480/3560 Training loss: 2.1843 0.0442 sec/batch
Epoch 3/20  Iteration 481/3560 Training loss: 2.1839 0.0415 sec/batch
Epoch 3/20  Iteration 482/3560 Training loss: 2.1834 0.0399 sec/batch
Epoch 3/20  Iteration 483/3560 Training loss: 2.1830 0.0398 sec/batch
Epoch 3/20  Iteration 484/3560 Training loss: 2.1828 0.0421 sec/batch
Epoch 3/20  Iteration 485/3560 Training loss: 2.1824 0.0415 sec/batch
Epoch 3/20  Iteration 486/3560 Training loss: 2.1822 0.0389 sec/batch
Epoch 3/20  Iteration 487/3560 Training loss: 2.1818 0.0392 sec/batch
Epoch 3/20  Iteration 488/3560 Training loss: 2.1812 0.0394 sec/batch
Epoch 3/20  Iteration 489/3560 Training loss: 2.1808 0.0439 sec/batch
Epoch 3/20  Iteration 490/3560 Training loss: 2.1806 0.0390 sec/batch
Epoch 3/20  Iteration 491/3560 Training loss: 2.1801 0.0391 sec/batch
Epoch 3/20  Iteration 492/3560 Training loss: 2.1798 0.0389 sec/batch
Epoch 3/20  Iteratio

...

Epoch 5/20  Iteration 724/3560 Training loss: 2.0083 0.0474 sec/batch
Epoch 5/20  Iteration 725/3560 Training loss: 2.0082 0.0410 sec/batch
Epoch 5/20  Iteration 726/3560 Training loss: 2.0099 0.0401 sec/batch
Epoch 5/20  Iteration 727/3560 Training loss: 2.0092 0.0401 sec/batch
Epoch 5/20  Iteration 728/3560 Training loss: 2.0078 0.0449 sec/batch
Epoch 5/20  Iteration 729/3560 Training loss: 2.0072 0.0453 sec/batch
Epoch 5/20  Iteration 730/3560 Training loss: 2.0091 0.0411 sec/batch
Epoch 5/20  Iteration 731/3560 Training loss: 2.0091 0.0400 sec/batch
Epoch 5/20  Iteration 732/3560 Training loss: 2.0085 0.0408 sec/batch
Epoch 5/20  Iteration 733/3560 Training loss: 2.0077 0.0426 sec/batch
Epoch 5/20  Iteration 734/3560 Training loss: 2.0096 0.0403 sec/batch
Epoch 5/20  Iteration 735/3560 Training loss: 2.0088 0.0401 sec/batch
Epoch 5/20  Iteration 736/3560 Training loss: 2.0078 0.0405 sec/batch
Epoch 5/20  Iteration 737/3560 Training loss: 2.0072 0.0414 sec/batch
Epoch 5/20  Iteratio

Epoch 5/20  Iteration 844/3560 Training loss: 1.9704 0.0430 sec/batch
Epoch 5/20  Iteration 845/3560 Training loss: 1.9703 0.0407 sec/batch
Epoch 5/20  Iteration 846/3560 Training loss: 1.9702 0.0401 sec/batch
Epoch 5/20  Iteration 847/3560 Training loss: 1.9700 0.0408 sec/batch
Epoch 5/20  Iteration 848/3560 Training loss: 1.9699 0.0403 sec/batch
Epoch 5/20  Iteration 849/3560 Training loss: 1.9698 0.0425 sec/batch
Epoch 5/20  Iteration 850/3560 Training loss: 1.9697 0.0406 sec/batch
Epoch 5/20  Iteration 851/3560 Training loss: 1.9698 0.0459 sec/batch
Epoch 5/20  Iteration 852/3560 Training loss: 1.9695 0.0405 sec/batch
Epoch 5/20  Iteration 853/3560 Training loss: 1.9695 0.0402 sec/batch
Epoch 5/20  Iteration 854/3560 Training loss: 1.9693 0.0425 sec/batch
Epoch 5/20  Iteration 855/3560 Training loss: 1.9691 0.0408 sec/batch
Epoch 5/20  Iteration 856/3560 Training loss: 1.9690 0.0405 sec/batch
Epoch 5/20  Iteration 857/3560 Training loss: 1.9687 0.0406 sec/batch
Epoch 5/20  Iteratio

Epoch 6/20  Iteration 964/3560 Training loss: 1.9144 0.0478 sec/batch
Epoch 6/20  Iteration 965/3560 Training loss: 1.9142 0.0458 sec/batch
Epoch 6/20  Iteration 966/3560 Training loss: 1.9145 0.0494 sec/batch
Epoch 6/20  Iteration 967/3560 Training loss: 1.9143 0.0427 sec/batch
Epoch 6/20  Iteration 968/3560 Training loss: 1.9144 0.0404 sec/batch
Epoch 6/20  Iteration 969/3560 Training loss: 1.9138 0.0408 sec/batch
Epoch 6/20  Iteration 970/3560 Training loss: 1.9136 0.0402 sec/batch
Epoch 6/20  Iteration 971/3560 Training loss: 1.9130 0.0409 sec/batch
Epoch 6/20  Iteration 972/3560 Training loss: 1.9130 0.0452 sec/batch
Epoch 6/20  Iteration 973/3560 Training loss: 1.9124 0.0420 sec/batch
Epoch 6/20  Iteration 974/3560 Training loss: 1.9121 0.0412 sec/batch
Epoch 6/20  Iteration 975/3560 Training loss: 1.9115 0.0421 sec/batch
Epoch 6/20  Iteration 976/3560 Training loss: 1.9110 0.0402 sec/batch
Epoch 6/20  Iteration 977/3560 Training loss: 1.9108 0.0400 sec/batch
Epoch 6/20  Iteratio

Epoch 7/20  Iteration 1083/3560 Training loss: 1.8668 0.0417 sec/batch
Epoch 7/20  Iteration 1084/3560 Training loss: 1.8653 0.0401 sec/batch
Epoch 7/20  Iteration 1085/3560 Training loss: 1.8649 0.0466 sec/batch
Epoch 7/20  Iteration 1086/3560 Training loss: 1.8669 0.0492 sec/batch
Epoch 7/20  Iteration 1087/3560 Training loss: 1.8670 0.0459 sec/batch
Epoch 7/20  Iteration 1088/3560 Training loss: 1.8671 0.0479 sec/batch
Epoch 7/20  Iteration 1089/3560 Training loss: 1.8667 0.0446 sec/batch
Epoch 7/20  Iteration 1090/3560 Training loss: 1.8686 0.0507 sec/batch
Epoch 7/20  Iteration 1091/3560 Training loss: 1.8678 0.0474 sec/batch
Epoch 7/20  Iteration 1092/3560 Training loss: 1.8671 0.0560 sec/batch
Epoch 7/20  Iteration 1093/3560 Training loss: 1.8667 0.0466 sec/batch
Epoch 7/20  Iteration 1094/3560 Training loss: 1.8654 0.0544 sec/batch
Epoch 7/20  Iteration 1095/3560 Training loss: 1.8642 0.0433 sec/batch
Epoch 7/20  Iteration 1096/3560 Training loss: 1.8643 0.0420 sec/batch
Epoch 

Epoch 7/20  Iteration 1202/3560 Training loss: 1.8403 0.0416 sec/batch
Epoch 7/20  Iteration 1203/3560 Training loss: 1.8402 0.0475 sec/batch
Epoch 7/20  Iteration 1204/3560 Training loss: 1.8402 0.0446 sec/batch
Epoch 7/20  Iteration 1205/3560 Training loss: 1.8401 0.0481 sec/batch
Epoch 7/20  Iteration 1206/3560 Training loss: 1.8401 0.0458 sec/batch
Epoch 7/20  Iteration 1207/3560 Training loss: 1.8403 0.0416 sec/batch
Epoch 7/20  Iteration 1208/3560 Training loss: 1.8401 0.0430 sec/batch
Epoch 7/20  Iteration 1209/3560 Training loss: 1.8402 0.0420 sec/batch
Epoch 7/20  Iteration 1210/3560 Training loss: 1.8400 0.0442 sec/batch
Epoch 7/20  Iteration 1211/3560 Training loss: 1.8399 0.0425 sec/batch
Epoch 7/20  Iteration 1212/3560 Training loss: 1.8399 0.0438 sec/batch
Epoch 7/20  Iteration 1213/3560 Training loss: 1.8396 0.0445 sec/batch
Epoch 7/20  Iteration 1214/3560 Training loss: 1.8397 0.0419 sec/batch
Epoch 7/20  Iteration 1215/3560 Training loss: 1.8397 0.0479 sec/batch
Epoch 

In [29]:
tf.train.get_checkpoint_state('checkpoints/anna')

model_checkpoint_path: "checkpoints/anna/i178_l512_2.463.ckpt"
all_model_checkpoint_paths: "checkpoints/anna/i178_l512_2.463.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [30]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [31]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [32]:
checkpoint = "checkpoints/anna/i3560_l512_1.122.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/anna/i3560_l512_1.122.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoints/anna/i3560_l512_1.122.ckpt
	 [[Node: save/RestoreV2_5 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_5/tensor_names, save/RestoreV2_5/shape_and_slices)]]
	 [[Node: save/RestoreV2_6/_37 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_80_save/RestoreV2_6", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_5', defined at:
  File "/home/luis/anaconda2/envs/dlnd_gpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/luis/anaconda2/envs/dlnd_gpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/luis/anaconda2/envs/dlnd_gpu/lib/python3.6/site-pac

In [ ]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

In [ ]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)